<a href="https://colab.research.google.com/github/pdiddy5550/VCFbuddy/blob/main/vcf_scratchV.2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas
import numpy
import requests
import re, os
import json
!pip install cyvcf2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Storing VCFs here: "drive/MyDrive/vcf_data/raw/sample.vcf"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Patrick's Work

In [ ]:
#Upload a file (remember to copy path in the following TOOL 1)
from google.colab import files
uploaded = files.upload()


Saving Myeloid-Panel.vcf to Myeloid-Panel.vcf


In [38]:
import cyvcf2
import requests
import json
import re
import xml.etree.ElementTree as ET

# Function to get ClinVar UID for a given rsID
def get_clinvar_uid(rs_number):
    esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        'db': 'clinvar',
        'term': rs_number,
        'retmode': 'json'
    }
    response = requests.get(esearch_url, params=params)
    print(f"esearch response status code: {response.status_code}")
    print(f"esearch response text: {response.text}")

    if response.status_code == 200 and response.text:
        try:
            data = response.json()
            uids = data['esearchresult']['idlist']
            return uids[0] if uids else None
        except json.JSONDecodeError:
            print("Failed to decode JSON from esearch response.")
            return None
    else:
        return None

# Function to query ClinVar using UID, ClinVar only gives XML out
def query_clinvar_with_rsid(rs_number):

    # Verify if a valid rsID is provided
    if rs_number is None or not rs_number.startswith("rs"):
        print("Invalid or no rsID provided.")
        return {}

    esummary_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    params = {
        'db': 'clinvar',
        'term': rs_number,  # Change from 'id' to 'term' to use rsID for querying
        'retmode': 'xml'    # Make sure to specify XML as the return mode
    }
    response = requests.get(esummary_url, params=params)

      # Process the response
    if response.status_code == 200 and response.content:
        try:
            root = ET.fromstring(response.content)
            variant_info = {}

            document_summary = root.find(".//DocumentSummary")
            if document_summary is not None:
                # Extract Variant Name
                title = document_summary.find(".//title")
                variant_info['Variant Name'] = title.text if title is not None else "Not Available"

                # Extract Clinical Significance
                clinical_significance = document_summary.find(".//clinical_significance/description")
                variant_info['Clinical Significance'] = clinical_significance.text if clinical_significance is not None else "Not Available"

                # Extract Review Status
                review_status = document_summary.find(".//clinical_significance/review_status")
                variant_info['Review Status'] = review_status.text if review_status is not None else "Not Available"

                # Extract Phenotypes
                phenotypes = document_summary.findall(".//trait_set/trait/trait_name")
                phenotype_names = [phenotype.text for phenotype in phenotypes if phenotype.text is not None]
                variant_info['Phenotype'] = phenotype_names if phenotype_names else "Not Available"

            return variant_info

        except ET.ParseError:
            print("Failed to parse XML from esummary response.")
            return {}
    else:
        print(f"Failed to retrieve data for UID {uid}, Response status: {response.status_code}")
        return {}



# Modified function to query ClinVar
def query_clinvar(variant_id):
    if variant_id is None:
        print("No variant ID provided.")
        return None

    rs_number = None
    if 'rs' in variant_id:
        rs_match = re.search(r'rs\d+', variant_id)
        if rs_match:
            rs_number = rs_match.group()

    if rs_number:
        uid = get_clinvar_uid(rs_number)
        if uid:
            return query_clinvar_with_uid(uid)
        else:
            print(f"No UID found for variant {rs_number}")
            return None
    else:
        print(f"No rs number found for variant {variant_id}")
        return None

# Function to extract metadata
def extract_metadata(vcf_reader):
    metadata = {}
    header_lines = vcf_reader.raw_header.split('\n')
    for line in header_lines:
        if line.startswith('##'):
            try:
                key, value = re.match(r'##(.+?)=(.+)', line).groups()
                metadata[key] = value
            except AttributeError:
                pass  # Skip lines that don't match the pattern
    return metadata

# Function to read VCF file and process each variant
def read_vcf(file_path):
    vcf_reader = cyvcf2.VCF(file_path)

    # Extract metadata
    metadata = extract_metadata(vcf_reader)
    print("Metadata:")
    for key, value in metadata.items():
        print(f"{key}: {value}")
    print("\nVariant Data:")

    # Initialize a list to store variant data along with ClinVar information
    variant_data_list = []

    # Process each record (variant) in the VCF
    for record in vcf_reader:
        variant_id = record.ID
        clinvar_info = query_clinvar(variant_id)

        variant_info = {
            "Position": record.POS,
            "ID": variant_id,
            "Reference": record.REF,
            "Alternates": record.ALT,
            "Quality": record.QUAL,
            "Filter": record.FILTER,
            "ClinVar": clinvar_info
        }
        variant_data_list.append(variant_info)

    # Convert the list of variant data to JSON format
    json_data = json.dumps(variant_data_list, indent=4)
    print(json_data)

    # Optionally, save the JSON data to a file
    with open('variant_data.json', 'w') as json_file:
        json_file.write(json_data)

# Test with a known rsID that exists in ClinVar
test_id = 'rs180177042'  # A known ClinVar rsID
test_result = query_clinvar(test_id)
print("Test Result for rs180177042:")
print(test_result)

# Replace this path with the actual path to your VCF file
vcf_file_path = '/content/Myeloid-Panel.vcf'  # Make sure the file path is correct
read_vcf(vcf_file_path)


esearch response status code: 200
esearch response text: {"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"2","retmax":"2","retstart":"0","idlist":["162797","13981"],"translationset":[],"translationstack":[{"term":"rs180177042[All Fields]","field":"All Fields","count":"2","explode":"N"},"GROUP"],"querytranslation":"rs180177042[All Fields]"}}

Test Result for rs180177042:
{'Variant Name': 'NM_001374258.1(BRAF):c.2034T>G (p.Asp678Glu)', 'Clinical Significance': 'Pathogenic', 'Review Status': 'criteria provided, multiple submitters, no conflicts', 'Phenotype': ['Inborn genetic diseases', 'Cardio-facio-cutaneous syndrome', 'not provided', 'RASopathy', 'Cardiofaciocutaneous syndrome 1']}
Metadata:
fileformat: VCFv4.2
FILTER: <ID=PASS,Description="All filters passed">
fileDate: 20231205
fileEncoding: windows-1252
source: CLC Genomics Server 23.0.5 build 20230915024236
reference: /CLC_References/homo_sapiens/sequence/hg19/Homo_sapiens_sequence_hg19
contig: <ID=MT,length=1

KeyboardInterrupt: ignored

# Anna's Work

In [ ]:
def parse_vcf_snp(filename):
    # This function parses SNPs from a VCF file
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('#'):
                print(line)
            else:
                match = re.match(r'^(.+?)\t(.+?)\t(.+?)\t(.+?)\t(.+?)\t', line)
                if match:
                    chrom, pos, tag, ref, alts = match.groups()
                    alts_list = alts.split(',')
                    snp = all(len(alt) == len(ref) for alt in alts_list) and len(ref) == 1

                    if snp:
                        print(line)

def parse_vcf_indels(filename):
    # This function parses indels from a VCF file
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('#'):
                print(line)
            else:
                match = re.match(r'^(.+?)\t(.+?)\t(.+?)\t(.+?)\t(.+?)\t', line)
                if match:
                    chrom, pos, tag, ref, alts = match.groups()
                    alts_list = alts.split(',')
                    snp = all(len(alt) == len(ref) for alt in alts_list)

                    if not snp:
                        print(line)

def parse_vcf_strip_headers(filename):

    seen_non_header = False
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if not seen_non_header:
                if line.startswith('#'):
                    pass
                else:
                    seen_non_header = True
                print(line)
            else:
                if not line.startswith('#'):
                    print(line)


In [ ]:
print(os.getcwd())
print(os.listdir())
filename = "drive/MyDrive/vcf_data/raw/sample.vcf"  # replace with your filename
parse_vcf_snp(filename)
# parse_vcf_indels(filename)
# parse_vcf_strip_headers(filename)

/content
['.config', 'drive', '.ipynb_checkpoints', 'data', 'sample_data']
##fileformat=VCFv4.0
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=1000GenomesPilot-NCBI36
##phasing=partial
##INFO=<ID=NS,Number=1,Type=Integer,Description="Number of Samples With Data">
##INFO=<ID=AN,Number=1,Type=Integer,Description="Total number of alleles in called genotypes">
##INFO=<ID=AC,Number=.,Type=Integer,Description="Allele count in genotypes, for each ALT allele, in the same order as listed">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Total Depth">
##INFO=<ID=AF,Number=.,Type=Float,Description="Allele Frequency">
##INFO=<ID=AA,Number=1,Type=String,Description="Ancestral Allele">
##INFO=<ID=DB,Number=0,Type=Flag,Description="dbSNP membership, build 129">
##INFO=<ID=H2,Number=0,Type=Flag,Description="HapMap2 membership">
##FILTER=<ID=q10,Description="Quality below 10">
##FILTER=<ID=s50,Description="Less than 50% of samples have data">
##FORMAT=<ID=GT,Number=1,Type=String,Desc

Mounted at /content/drive


# ClinVar

In [ ]:
# import subprocess
# import pysam

In [ ]:
# !pip install pysam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 61.3 MB/s eta 0:00:00


In [ ]:
# def search_clinvar(rs_id):
#     try:
#         esearch_process = subprocess.run(
#             ["esearch", "-db", "clinvar", "-query", rs_id],
#             stdout=subprocess.PIPE,
#             check=True,
#         )
#         webenv = None
#         query_key = None
#         for line in esearch_process.stdout.decode().split("\n"):
#             if line.startswith("<WebEnv>"):
#                 webenv = line[len("<WebEnv>"):-len("</WebEnv>")]
#             elif line.startswith("<QueryKey>"):
#                 query_key = line[len("<QueryKey>"):-len("</QueryKey>")]

#         if webenv and query_key:
#             efetch_process = subprocess.run(
#                 [
#                     "efetch",
#                     "-db",
#                     "clinvar",
#                     "-format",
#                     "xml",
#                     "-WebEnv",
#                     webenv,
#                     "-query_key",
#                     query_key,
#                 ],
#                 stdout=subprocess.PIPE,
#                 check=True,
#             )
#             return efetch_process.stdout.decode()
#     except subprocess.CalledProcessError:
#         return None


# def parse_vcf_and_search(input_vcf_file):
#     vcf_reader = pysam.VariantFile(input_vcf_file)
#     for record in vcf_reader:
#         rs_id = record.id  # Assuming the ID field contains the RS number
#         if rs_id and rs_id.startswith("rs"):  # Check if id looks like a valid RS number
#             print(f"Searching for {rs_id} in ClinVar")
#             clinvar_data = search_clinvar(rs_id)
#             if clinvar_data:
#                 print(f"Information for {rs_id}:\n{clinvar_data}")
#             else:
#                 print(f"{rs_id} not found in ClinVar")


In [ ]:
# input_vcf_file = 'data/raw/sample.vcf'
# parse_vcf_and_search(input_vcf_file)

Searching for rs6054257 in ClinVar


FileNotFoundError: ignored